In [ ]:
import requests
import io
url='https://cernbox.cern.ch/remote.php/dav/public-files/FbXw3V4XNyYB3oA/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5'
with requests.Session() as session:
    r = session.get(url, stream=True)
    r.raise_for_status()
    with open('SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5', 'wb') as hd5:
        for chunk in r.iter_content(chunk_size=io.DEFAULT_BUFFER_SIZE):
            hd5.write(chunk)

In [ ]:
url='https://cernbox.cern.ch/remote.php/dav/public-files/AtBT8y4MiQYFcgc/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5'
with requests.Session() as session:
    r = session.get(url, stream=True)
    r.raise_for_status()
    with open('SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5', 'wb') as hd5:
        for chunk in r.iter_content(chunk_size=io.DEFAULT_BUFFER_SIZE):
            hd5.write(chunk)

In [ ]:
import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, transforms
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve

In [ ]:
data_electron = h5py.File('/content/SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5')
data_photon = h5py.File('/content/SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5')

In [ ]:
datasets = [data_electron, data_photon]
X = np.concatenate([data['/X'][:16000] for data in datasets])
y = np.concatenate([data['/y'][:16000] for data in datasets])

In [ ]:
arr2_reshaped = np.expand_dims(X[:, :, :, 0], axis=-1)
result = np.concatenate([X, arr2_reshaped], axis=-1)
result.shape

(32000, 32, 32, 3)

In [ ]:
result = np.moveaxis(result, 1, -1)
result.shape

(32000, 3, 32, 32)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(result, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=35)

In [ ]:
print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)

(23040, 3, 32, 32)
(2560, 3, 32, 32)
(6400, 3, 32, 32)
(23040,)


In [ ]:
from torch.utils.data import DataLoader, TensorDataset
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_train = y_train.type(torch.LongTensor)
train_dataset = TensorDataset(X_train, y_train)
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)
y_val = y_val.type(torch.LongTensor)
val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

In [ ]:
X_test = torch.tensor(X_train, dtype=torch.float32)
y_test = torch.tensor(y_train, dtype=torch.float32)
y_test = y_test.type(torch.LongTensor)
test_dataset = TensorDataset(X_train, y_train)
testloader = DataLoader(test_dataset, batch_size=32)

<ipython-input-12-f80108c13f3c>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_test = torch.tensor(X_train, dtype=torch.float32)
<ipython-input-12-f80108c13f3c>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_train, dtype=torch.float32)


In [ ]:
device = torch.device('cuda')

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU(inplace=True)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU(inplace=True)
        
        self.fc1 = nn.Linear(in_features=128 * 8 * 8, out_features=256)
        self.relu4 = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(in_features=256, out_features=2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = self.relu3(x)
        
        x = x.view(-1, 128 * 8 * 8)
        
        x = self.fc1(x)
        x = self.relu4(x)
        x = self.fc2(x)
        return x


net = Net()
if torch.cuda.is_available():
    net.cuda()

In [ ]:
def calculate_loss_and_accuracy(model, dataloader, size_of_dataset, criterion):
    
    # Now set model to validation mode.
    running_loss = 0
    running_accuracy = 0
    
     # Processing the Test Loader
    for (inputs, labels) in dataloader:
        
        # Load data to device.
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _ , preds = torch.max(outputs, 1)
        
        # Outputs
        outputs = model(inputs)
        _ , preds = torch.max(outputs, 1)
        
        # Loss and Backpropagation.
        loss = criterion(outputs, labels)
        
        # Statistics
        running_loss += loss.item()*inputs.size(0)
        running_accuracy += torch.sum(preds == labels.data)
        
    epoch_loss = running_loss/size_of_dataset
    epoch_accuracy = running_accuracy/size_of_dataset
    
    return epoch_loss, epoch_accuracy

In [ ]:
import copy

def train(model, criterion, optimizer, scheduler, num_of_epochs):
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    #track_training_loss = []
    #track_val_loss = []

    for epoch in range(num_of_epochs):

        print(f'\nEpoch {epoch + 1}/{num_of_epochs}')
        print('-'*30)

        model.train() # Setting model to train.
        running_loss = 0
        running_accuracy = 0

        # Processing the Train Loader
        for (inputs, labels) in trainloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad() # zero the parameter gradients

            # Outputs
            outputs = model(inputs)
            _ , preds = torch.max(outputs, 1)

            # Loss and Backpropagation.
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # Statistics
            running_loss += loss.item()*inputs.size(0)
            running_accuracy += torch.sum(preds == labels.data)
        
        scheduler.step()
        epoch_loss = running_loss/len(train_dataset)
        epoch_accuracy = running_accuracy/len(train_dataset)
        #track_training_loss.append(epoch_loss) # Loss Tracking
        print(f'Training Loss: {epoch_loss:.4f} Training Acc.: {epoch_accuracy:.4f}')

        # Now set model to validation mode.
        model.eval()

        val_loss, val_accuracy = calculate_loss_and_accuracy(model, val_loader, len(val_dataset), criterion)

        if val_accuracy > best_acc:
            print("Found better model...")
            print('Updating the model weights....\n')
            print(f'Val Loss: {val_loss:.4f} Val Acc.: {val_accuracy:.4f}\n')

            best_acc = val_accuracy
            best_model_wts = copy.deepcopy(model.state_dict())
     
    model.load_state_dict(best_model_wts) # update model
    
    return  model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

NUM_OF_EPOCHS = 10
best_model = train(model = net,
                   criterion = criterion,
                   optimizer = optimizer,
                   scheduler = exp_lr_scheduler,
                   num_of_epochs = NUM_OF_EPOCHS
                  )
filepath = "/content/modelModule.pt"
torch.save(net.state_dict(), filepath)


Epoch 1/10
------------------------------
Training Loss: 0.6931 Training Acc.: 0.5060
Found better model...
Updating the model weights....

Val Loss: 0.6931 Val Acc.: 0.4938


Epoch 2/10
------------------------------
Training Loss: 0.6929 Training Acc.: 0.5119

Epoch 3/10
------------------------------
Training Loss: 0.6927 Training Acc.: 0.5151
Found better model...
Updating the model weights....

Val Loss: 0.6923 Val Acc.: 0.5063


Epoch 4/10
------------------------------
Training Loss: 0.6923 Training Acc.: 0.5223

Epoch 5/10
------------------------------
Training Loss: 0.6920 Training Acc.: 0.5370
Found better model...
Updating the model weights....

Val Loss: 0.6914 Val Acc.: 0.5711


Epoch 6/10
------------------------------
Training Loss: 0.6914 Training Acc.: 0.5416

Epoch 7/10
------------------------------
Training Loss: 0.6904 Training Acc.: 0.5492
Found better model...
Updating the model weights....

Val Loss: 0.6893 Val Acc.: 0.5926


Epoch 8/10
-----------------------